<a href="https://colab.research.google.com/github/sinux-l5d/PROJ002/blob/main/animal_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from PIL import Image
from dataclasses import dataclass
from pathlib import Path
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

In [4]:
DATADIR = "animal10"
WIDTH = 200
HEIGHT = 200

In [3]:
data_zip = "/content/drive/MyDrive/INFO002/animal-10.zip"
with zipfile.ZipFile(data_zip,"r") as zip_ref:
    zip_ref.extractall(DATADIR)

In [5]:
#run on graphic card if possible
gpus = tf.config.list_physical_devices('GPU')
print("nb gpus", len(gpus))
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

nb gpus 1
1 Physical GPUs, 1 Logical GPU


In [6]:
translate = {'cane': 'dog', 'cavallo': 'horse', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'pecora': 'sheep', 'scoiattolo': 'squirrel', 'dog': 'cane', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'spider': 'ragno', 'squirrel': 'scoiattolo', 'horse': 'cavallo', 'ragno': 'spider', 'sheep': 'pecora'}
CATEGORIES = [translate[name] for name in listdir(DATADIR + "/raw-img")]
CATEGORIES.sort()
print(list(enumerate(CATEGORIES)))

[(0, 'butterfly'), (1, 'cat'), (2, 'chicken'), (3, 'cow'), (4, 'dog'), (5, 'elephant'), (6, 'horse'), (7, 'sheep'), (8, 'spider'), (9, 'squirrel')]


In [46]:
# dataclass
# class Photo:
#   path: str
#   animal: str

#   def load(self):
#     return Image.open(self.path).convert("RGB").resize((WIDTH,HEIGHT))

#   def variations(self):
#     img = self.load()
#     return [img.transpose(Image.FLIP_LEFT_RIGHT),
#             img.transpose(Image.FLIP_TOP_BOTTOM),
#             img.transpose(Image.ROTATE_90),
#             img.resize((50,50)).resize((WIDTH,HEIGHT))]
#   def tagI(self) -> int:
#     return CATEGORIES.index(self.animal)
#   def tagS(self) -> str:
#     return self.animal


@dataclass
class Collection:
  path = Path(DATADIR + "/raw-img")
  train_perct = 0.8

  def getCategory(self, category: str) -> pd.Series:
    dirsp = translate.get(category, None)
    if dirsp == None:
      raise Exception("Invalid category")
    imgs = list(self.path.glob(dirsp + "/*"))
    filenames = pd.Series(imgs, name="Filepath").astype(str)
    labels = pd.Series([category for _ in range(len(imgs))], name="Label")
    return pd.concat([filenames, labels], axis=1)

  def computeMaxs(self, maxreq):
    lenghts = {k: len(self.getCategory(k)) for k in CATEGORIES}
    total = sum(lenghts.values())
    return {k: round((v/total)*maxreq) for k, v in lenghts.items()}

  def getTrain(self, maxs={}):
      out = []
      for category in CATEGORIES:
          df = self.getCategory(category)
          end = maxs.get(category, len(df))
          end = round(end * self.train_perct)
          out.append(df[:end])
      return pd.concat(out)

  def getTest(self, maxs={}):
      out = []
      for category in CATEGORIES:
          df = self.getCategory(category)
          end = maxs.get(category, len(df))
          start = round(end * self.train_perct)
          out.append(df[start:end])
      return pd.concat(out)


In [8]:
def display_row(*imgs):
    _, ax = plt.subplots(1, len(imgs), figsize=(20, 20))
    if not isinstance(ax, np.ndarray):
        ax = np.array([ax])
    for i, img in enumerate(imgs):
        ax[i].imshow(img)
        ax[i].axis('off')

In [49]:
c = Collection()
#display_row(*c.getTrain()[2000].variations())

m = c.computeMaxs(10_000)
print(m)
assert sum(m.values()) == 10_000
assert len(c.getTrain(m)) + len(c.getTest(m)) == 10_000
print(c.getTrain(m))
del m # was used in previous version

{'butterfly': 807, 'cat': 637, 'chicken': 1183, 'cow': 713, 'dog': 1858, 'elephant': 552, 'horse': 1002, 'sheep': 695, 'spider': 1842, 'squirrel': 711}
                                              Filepath      Label
0    animal10/raw-img/farfalla/OIP-HNpo_i1VSZXY9NQ3...  butterfly
1    animal10/raw-img/farfalla/OIP-6Bam26g68QXMl0z2...  butterfly
2    animal10/raw-img/farfalla/OIP-0nCXiQiaWGKe08X6...  butterfly
3    animal10/raw-img/farfalla/OIP-kkshTVubNjIsktE2...  butterfly
4    animal10/raw-img/farfalla/OIP-cilySFIjrc6QSJBe...  butterfly
..                                                 ...        ...
564  animal10/raw-img/scoiattolo/OIP-DLuAMgbGGOGW0r...   squirrel
565  animal10/raw-img/scoiattolo/OIP-BFCOTEmVj1I9R7...   squirrel
566  animal10/raw-img/scoiattolo/OIP-adLOsEpklYQOL5...   squirrel
567  animal10/raw-img/scoiattolo/OIP-sMWZFznxn4dnm_...   squirrel
568  animal10/raw-img/scoiattolo/OIP-auuw2Tn5WM35wa...   squirrel

[8001 rows x 2 columns]


In [65]:
from keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator(rescale = 1./255,
                                  width_shift_range=0.15,
                                   height_shift_range=0.15,
                                   horizontal_flip=True,
                                   vertical_flip=True)
datagen_test = ImageDataGenerator(rescale=1./255)

In [66]:
def getGenerator(idg, df, batch_size=32):
    return idg.flow_from_dataframe(
        dataframe=df,
        x_col="Filepath",
        y_col="Label",
        target_size=(WIDTH, HEIGHT),
        batch_size=batch_size,
        shuffle=True,
        class_mode="categorical"
    )

In [67]:
set_train = getGenerator(datagen_train, c.getTrain())
set_test = getGenerator(datagen_test, c.getTest())
# set_test_blind = getGenerator(datagen_test, c.getTest()["Filepath"])

Found 20943 validated image filenames belonging to 10 classes.
Found 5236 validated image filenames belonging to 10 classes.


In [96]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(WIDTH, HEIGHT, 3)),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(WIDTH, HEIGHT, 3)),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 198, 198, 64)      1792      
                                                                 
 batch_normalization_11 (Ba  (None, 198, 198, 64)      256       
 tchNormalization)                                               
                                                                 
 dropout_4 (Dropout)         (None, 198, 198, 64)      0         
                                                                 
 conv2d_14 (Conv2D)          (None, 196, 196, 32)      18464     
                                                                 
 batch_normalization_12 (Ba  (None, 196, 196, 32)      128       
 tchNormalization)                                               
                                                                 
 dropout_5 (Dropout)         (None, 196, 196, 32)    

In [97]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best.keras", monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")
]

model.fit(set_train,
          validation_data = set_test,
          validation_steps = len(set_test),
          epochs=10,
          callbacks=callbacks,
          batch_size=32)

Epoch 1/10
655/655 [==============================] - ETA: 0s - loss: 30.4263 - accuracy: 0.1836
Epoch 1: val_accuracy improved from -inf to 0.19099, saving model to best.keras
655/655 [==============================] - 359s 535ms/step - loss: 30.4263 - accuracy: 0.1836 - val_loss: 2.2396 - val_accuracy: 0.1910
Epoch 2/10
655/655 [==============================] - ETA: 0s - loss: 2.2718 - accuracy: 0.1959
Epoch 2: val_accuracy improved from 0.19099 to 0.19882, saving model to best.keras
655/655 [==============================] - 357s 544ms/step - loss: 2.2718 - accuracy: 0.1959 - val_loss: 2.2943 - val_accuracy: 0.1988
Epoch 3/10
655/655 [==============================] - ETA: 0s - loss: 2.1997 - accuracy: 0.2162
Epoch 3: val_accuracy improved from 0.19882 to 0.22326, saving model to best.keras
655/655 [==============================] - 357s 544ms/step - loss: 2.1997 - accuracy: 0.2162 - val_loss: 2.2662 - val_accuracy: 0.2233
Epoch 4/10
655/655 [==============================] - ETA: 

In [99]:
model.load_weights("best.keras")
test_loss, test_acc = model.evaluate(set_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

164/164 [==============================] - 11s 66ms/step - loss: 1.9829 - accuracy: 0.3052
Test accuracy: 0.30519479513168335
Test loss: 1.9829366207122803


In [ ]:
# pred = model.predict(set_test_blind)

# pred[0]